<a href="https://colab.research.google.com/github/Marin-kh/Persian_RAG/blob/main/Persian_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-docx
!pip install rake_nltk
!pip install docx
!pip install stanza
!pip install gradio
!pip install langchain_groq
!pip install langgraph langchain_openai langchain_core
!pip install langchain_community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docx: filename=docx-0.2.4-py3-none-any.whl size=53893 sha256=6906acdb6d7b41720daa15c01fad0c0c9c7ab45a5d75f7cbf60eb4fc4bb16c90
  Stored in directory: /root/.cache/pip/wheels/c1/3e/c3/e81c11effd0be5658a035947c66792dd993bcff317eae0e1ed
Successfully built docx
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 770.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [ ]:
!pip install hazm

  Using cached numpy-1.24.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
Using cached numpy-1.24.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.5
    Uninstalling numpy-2.2.5:
      Successfully uninstalled numpy-2.2.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.3.23 requires numpy>=1.26.2; python_version < "3.13", but you have numpy 1.24.3 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.3 which is incompatible.
pymc 5.22.0 requires numpy>=1.25.0, but you have numpy 1.24.3 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.24.3 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.24.3 which is incompat

In [ ]:
import numpy as np
import pandas as pd
from hazm import stopwords_list, Normalizer, WordTokenizer, SentenceTokenizer, Stemmer, Lemmatizer, sent_tokenize, word_tokenize
import docx
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import requests
from sklearn.metrics.pairwise import cosine_similarity
import openai
import nltk
from rake_nltk import Rake
from google.colab import drive
import stanza
from collections import defaultdict
from openai import OpenAI
from typing import Optional, TypedDict, Annotated
from langchain_core.tools import tool, StructuredTool
from langchain_groq import ChatGroq
from langchain.llms import OpenAI
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph
from langchain_core.messages import SystemMessage, AnyMessage, HumanMessage, ToolMessage
import operator
import gradio as gr
import json

In [ ]:
stanza.download('fa')
nlp = stanza.Pipeline('fa')

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: fa (Persian) ...


INFO:stanza:Downloaded file to /root/stanza_resources/fa/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: fa (Persian):
| Processor | Package        |
------------------------------
| tokenize  | perdt          |
| mwt       | perdt          |
| pos       | perdt_charlm   |
| lemma     | perdt_nocharlm |
| depparse  | perdt_charlm   |
| ner       | arman          |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [ ]:
class PersianRAKE(Rake):
    def _tokenize_text_to_sentences(self, text: str):
        return sent_tokenize(text)

    def _tokenize_sentence_to_words(self, sentence: str):
        return word_tokenize(sentence)

In [ ]:
def read_from_docx(doc):
    fullText=''
    for pra in doc.paragraphs:
        fullText+=pra.text+' '

    return fullText

def split_into_overlapping_chunks(sentences, max_chunk_size=1000, overlap_size=200):
    chunks = []
    current_chunk = ""
    current_chunk_size = 0

    for sentence in sentences:
        sentence_length = len(sentence)

        if current_chunk_size + sentence_length > max_chunk_size and current_chunk:
            chunks.append(current_chunk.strip())

            overlap_buffer = current_chunk[-overlap_size:].strip() if current_chunk else ""
            current_chunk = overlap_buffer + " "
            current_chunk_size = len(overlap_buffer) + 1

        current_chunk += sentence + " "
        current_chunk_size += sentence_length + 1

    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks

def preprocess_text_1(text):
    # text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'( +)', ' ', str(text))
    return text


def preprocess_text_2(text):
    text = re.sub('(\(.*?\))|(\[.*?\])', '', str(text))
    text = re.sub(r'( +)', ' ', str(text))

    word_tokenizer = WordTokenizer()
    words = word_tokenizer.tokenize(text)

    stopwords = stopwords_list()
    filtered_words = [word for word in words if word not in stopwords]

    lemmatizer = Lemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]

    return ' '.join(lemmatized_words)


def check_spelling(main_text):
    endpoint = "https://api.languagetool.org/v2/check"

    data = {
        "text": main_text,
        "language": "en-US",
    }

    response = requests.post(endpoint, data=data)
    json_response = response.json()

    updated_text = main_text

    for match in json_response.get("matches", []):
        replacement = match["replacements"][0]["value"] if match["replacements"] else ""

        offset = match["offset"]
        length = match["length"]

        updated_text = updated_text.replace(main_text[offset:offset+length], replacement)

    print("Original Query: ", main_text)
    print("Spell-checked Query: ", updated_text)
    return updated_text

def phrase_search(sentence):
    doc = nlp(sentence)

    phrases = []
    for sent in doc.sentences:
        for word in sent.words:
            if word.upos in ['NOUN', 'ADJ']:
                phrase = word.text
                for other_word in sent.words:
                    if other_word.head == word.id and other_word.upos in ['NOUN', 'ADJ']:
                        phrase += " " + other_word.text
                if " " in phrase:
                    phrases.append(phrase)
    return phrases

def english_to_persian_number(number_str):
    english_to_persian = {
        "0": "۰",
        "1": "۱",
        "2": "۲",
        "3": "۳",
        "4": "۴",
        "5": "۵",
        "6": "۶",
        "7": "۷",
        "8": "۸",
        "9": "۹",
    }
    persian_number = "".join([english_to_persian[digit] for digit in number_str])
    return persian_number

def persian_words_to_number(sentence):
    word_to_number = {
        "صفر": 0,
        "یک": 1,
        "دو": 2,
        "سه": 3,
        "چهار": 4,
        "پنج": 5,
        "شش": 6,
        "هفت": 7,
        "هشت": 8,
        "نه": 9,
        "ده": 10,
        "یازده": 11,
        "دوازده": 12,
        "سیزده": 13,
        "چهارده": 14,
        "پانزده": 15,
        "شانزده": 16,
        "هفده": 17,
        "هجده": 18,
        "نوزده": 19,
        "بیست": 20,
        "سی": 30,
        "چهل": 40,
        "پنجاه": 50,
        "شصت": 60,
        "هفتاد": 70,
        "هشتاد": 80,
        "نود": 90,
        "صد": 100,
        "یکصد": 100,
        "دویست": 200,
        "سیصد": 300,
        "چهارصد": 400,
        "پانصد": 500,
        "ششصد": 600,
        "هفتصد": 700,
        "هشتصد": 800,
        "نهصد": 900,
        "هزار": 1000,
    }
    words = sentence.split(' ')

    result = []
    temp_number_words = []
    current_number = 0

    for word in words:
        if word[-1:]=='م' and (word[:-1] in word_to_number):
            word = word[:-1]
        if word in word_to_number:
            temp_number_words.append(word)
            current_number += word_to_number[word]
        else:
            if temp_number_words:
                english_number_str = str(current_number)
                persian_number_str = english_to_persian_number(english_number_str)
                result.append(persian_number_str)
                temp_number_words = []
                current_number = 0
            result.append(word)

    if temp_number_words:
        english_number_str = str(current_number)
        persian_number_str = english_to_persian_number(english_number_str)
        result.append(persian_number_str)

    return ' '.join(result)

def preprocess_phrases(text, phrases):
    for phrase in phrases:
        text = text.replace(phrase, phrase.replace(" ", "_"))
    return text

def extract_persian_numbers(text):
    persian_digits = "۰۱۲۳۴۵۶۷۸۹"
    return re.findall(f"[{persian_digits}]+", text)

def calculate_cosine_similarity(docs, phrase):
    vectorizer = TfidfVectorizer(ngram_range=(1, 3))
    tfidf_matrix = vectorizer.fit_transform(docs)
    phrase_vector = vectorizer.transform(phrase)
    return cosine_similarity(phrase_vector, tfidf_matrix)

def calculate_tf(document_numbers):
    tf = []
    for doc in document_numbers:
        tf_dict = defaultdict(int)
        for num in doc:
            tf_dict[num] += 1
        tf.append(tf_dict)
    return tf

def calculate_idf(document_numbers, numbers):
    idf = {}
    total_docs = len(document_numbers)
    for num in numbers:
        doc_count = sum(1 for doc in document_numbers if num in doc)
        idf[num] = np.log((total_docs + 1) / (doc_count + 1)) + 1
    return idf

def calculate_tf_idf(document_numbers, numbers):
    tf = calculate_tf(document_numbers)
    idf = calculate_idf(document_numbers, numbers)
    tf_idf = []
    for doc_tf in tf:
        doc_tf_idf = {}
        for num, freq in doc_tf.items():
            if num in idf:
                doc_tf_idf[num] = freq * idf[num]
        tf_idf.append(doc_tf_idf)
    return tf_idf

def get_city_link_by_name(persian_name: str):
    try:
        result = df_cities[df_cities['name'] == persian_name]['name_en']

        if not result.empty:
            return result.iloc[0]
        else:
            return None
    except Exception as e:
        return None

In [ ]:
# Loading The Main Document
drive.mount('/content/drive')
document = read_from_docx(docx.Document("/content/drive/My Drive/delta.docx"))
df_api = pd.read_csv('/content/drive/My Drive/api_key.csv')
df_cities = pd.read_csv('/content/drive/My Drive/cities_per_en.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
base_llm = ChatGroq(
    model="gemma2-9b-it",
    api_key=df_api.loc[2, 'api_key'],
    temperature=0.5
)

In [ ]:
# Chunking The Document
normalizer = Normalizer()
normalized_text = normalizer.normalize(document)

sentence_tokenizer = SentenceTokenizer()
sentences = sentence_tokenizer.tokenize(normalized_text)

max_chunk_size = 1000
overlap_size = 200
chunks = split_into_overlapping_chunks(sentences, max_chunk_size, overlap_size)
print("<Chunk 1>")
print(f"Original Chunk:\n{chunks[0]}")

# Preprocessing The Chunks
preprocessed1_chunks = [preprocess_text_1(chunk) for chunk in chunks]

preprocessed2_chunks = [preprocess_text_2(chunk) for chunk in preprocessed1_chunks]
print(f"Preprocessed Chunk:\n{preprocessed2_chunks[0]}")

<Chunk 1>
Original Chunk:
املاک دلتا یکی از شرکت‌های برجسته در صنعت املاک و مستغلات است که در زمینه خرید، فروش، اجاره و مدیریت املاک فعالیت می‌کند. این شرکت با بهره‌گیری از تجربه و تخصص خود در این حوزه، به مشتریان خود خدمات متنوعی ارائه می‌دهد. در این مقاله به بررسی تاریخچه، خدمات، مزایا و چالش‌های املاک دلتا خواهیم پرداخت: تاریخچه دلتا املاک دلتا در سال ۱۳۵۶ تأسیس شد و از آن زمان تا به امروز به یکی از نام‌های معتبر در صنعت املاک و مستغلات تبدیل‌شده است. این شرکت با هدف ارائه خدمات با کیفیت و ساختارهای نوآورانه به مشتریان، فعالیت خود را آغاز کرد. با گذشت زمان، املاک دلتا توانسته است با بهره‌گیری از تیمی متخصص و استفاده از فناوری‌های پیشرفته، جایگاه خود را در بازار مستحکم کند. خدمات املاک دلتا به ارائه طیف گسترده‌ای از خدمات در حوزه املاک و مستغلات می‌پردازد که شامل موارد زیر است: خرید و فروش املاک: این شرکت به مشتریان خود کمک می‌کند تا املاک مناسب برای خرید یا فروش را پیدا کنند. خدمات مشاوره‌ای در زمینه ارزیابی قیمت، معرفی املاک مناسب و فرآیندهای قانونی خرید و فروش از جمله خدمات این بخ

In [54]:
PROPERTY_TYPE_MAPPING = {
    "زمین": {"url": "store", "display": "زمین"},
    "آپارتمان موقعیت اداری": {"url": "OfficeLocationApartment", "display": "آپارتمان موقعیت اداری"},
    "باغ": {"url": "garden", "display": "باغ"},
    "باغچه": {"url": "garden", "display": "باغ"},
    "تجاری": {"url": "commercial", "display": "تجاری"},
    "مغازه": {"url": "commercial", "display": "مغازه"},
    "آپارتمان": {"url": "apartment", "display": "آپارتمان"}
}

def normalize_query(query: str) -> str:
    """Normalize user input to fix common typos."""
    query = query.replace("آجهي", "آگهی").replace("تند", "تعداد")
    query = query.replace("\u200c", " ")
    return query.strip()

def clean_tool_call(tool_call: dict) -> dict:
    """Clean and validate tool call data."""
    tool_name = tool_call.get("name", "").strip()
    tool_args = tool_call.get("args", {})

    if isinstance(tool_args, str):
        try:
            tool_args = json.loads(tool_args.replace("'", "\""))
        except json.JSONDecodeError:
            tool_args = {"query": tool_args} if tool_args else {}

    if not isinstance(tool_args, dict):
        tool_args = {"query": str(tool_args)}

    if "query" in tool_args:
        tool_args["query"] = normalize_query(tool_args["query"])

    return {"name": tool_name, "args": tool_args}

def parse_content_tool_call(content: str) -> Optional[dict]:
    """Parse a tool call from response.content if it matches the expected format."""
    pattern = r'<function=(\w+)\s*(\{.*?\})\s*</function>'
    match = re.match(pattern, content.strip())
    if match:
        tool_name = match.group(1)
        try:
            tool_args = json.loads(match.group(2))
            return {"name": tool_name, "args": tool_args}
        except json.JSONDecodeError:
            return None
    return None

def search_properties_func(min_meter: Optional[int] = None, max_meter: Optional[int] = None,
                         min_price: Optional[int] = None, max_price: Optional[int] = None,
                         city: Optional[str] = "tehran", elevator: Optional[bool] = None,
                         property_type: Optional[str] = "apartment") -> str:
    """
    جستجو بین املاک سایت با فیلتر شهر، متراژ، قیمت، آسانسور و نوع ملک.
    """
    if city:
        city_en = get_city_link_by_name(city)
        city = city_en if city_en else "tehran"
    else:
        city = "tehran"

    property_info = PROPERTY_TYPE_MAPPING.get(property_type, PROPERTY_TYPE_MAPPING["آپارتمان"])
    property_url = property_info["url"]
    property_display = property_info["display"]

    base_url = f"https://deltadev.ir/{city}/buy/{property_url}"

    query_params = []
    if min_meter is not None and max_meter is not None:
        query_params.append(f"meter={min_meter}-{max_meter}")
    elif min_meter is not None:
        query_params.append(f"meter={min_meter}-")
    elif max_meter is not None:
        query_params.append(f"meter=-{max_meter}")

    if min_price is not None and max_price is not None:
        query_params.append(f"price={min_price}-{max_price}")
    elif min_price is not None:
        query_params.append(f"price={min_price}-")
    elif max_price is not None:
        query_params.append(f"price=-{max_price}")

    if elevator:
        query_params.append("features=elevator")

    query_string = "?" + "&".join(query_params) if query_params else ""
    url = base_url + query_string

    filters = []
    if min_meter is not None and max_meter is not None:
        filters.append(f"{min_meter} تا {max_meter} متر")
    elif min_meter is not None:
        filters.append(f"حداقل {min_meter} متر")
    elif max_meter is not None:
        filters.append(f"حداکثر {max_meter} متر")

    if min_price is not None and max_price is not None:
        filters.append(f"بین {min_price:,} تا {max_price:,} تومان")
    elif min_price is not None:
        filters.append(f"حداقل {min_price:,} تومان")
    elif max_price is not None:
        filters.append(f"حداکثر {max_price:,} تومان")

    if elevator:
        filters.append("با آسانسور")

    if filters:
        filters_str = "، ".join(filters)
        explanation = f"با ویژگی‌هایی که گفتید ({filters_str})، می‌توانید بر روی این لینک کلیک کنید و ملک‌های مشابه را پیدا کنید: {url}"
    else:
        explanation = f"می‌توانید بر روی این لینک کلیک کنید و ملک‌های موجود را مشاهده کنید: {url}"

    return explanation

search_properties = StructuredTool.from_function(
    func=search_properties_func,
    name="search_properties",
    description="جستجوی ملک برای خرید با فیلترهای شهر، متراژ، قیمت، آسانسور، و نوع ملک.",
    args_schema=None
)

def extract_text_func(query: str) -> str:
    """
    استخراج و پردازش متن از داکیومنت با جستجوی هیبریدی و تولید پاسخ با LLM.
    """
    query = check_spelling(query)
    processed_query = preprocess_text_2(preprocess_text_1(query))
    query_n = persian_words_to_number(processed_query)
    phrases = phrase_search(query_n)
    numbers = re.findall(r'\d+', query_n)
    numberic_chunks = [persian_words_to_number(chunk) for chunk in preprocessed2_chunks]

    query_bonus = calculate_cosine_similarity(numberic_chunks, [query_n]).flatten()

    document_numbers = [extract_persian_numbers(doc) for doc in numberic_chunks]
    tf_idf_numbers = calculate_tf_idf(document_numbers, numbers)

    number_bonus = np.zeros(len(numberic_chunks))
    for i, doc_tf_idf in enumerate(tf_idf_numbers):
        for num in numbers:
            if num in doc_tf_idf:
                number_bonus[i] += doc_tf_idf[num]
    if np.max(number_bonus) > 0:
        number_bonus = number_bonus / np.max(number_bonus)

    preprocessed_docs = [preprocess_phrases(doc, phrases) for doc in numberic_chunks]
    preprocessed_phrases = preprocess_phrases(query_n, phrases)

    phrases_similarity = calculate_cosine_similarity(preprocessed_docs, [preprocessed_phrases])
    phrases_bonus = np.max(phrases_similarity, axis=0)
    if np.max(phrases_bonus) > 0:
        phrases_bonus = phrases_bonus / np.max(phrases_bonus)

    query_coef = 0.5
    numbers_coef = 0.3
    phrases_coef = 0.2

    hybrid_scores = (query_bonus * query_coef) + (number_bonus * numbers_coef) + (phrases_bonus * phrases_coef)

    top_k = 3
    indices = np.argsort(-hybrid_scores)[:top_k]

    top_3 = [chunks[idx] for idx in indices]
    while len(top_3) < 3:
        top_3.append("")

    llm = base_llm

    prompt = f"""
    {top_3[0]}\n{top_3[1]}\n{top_3[2]}
    طبق اطلاعات بالا، به طور خلاصه به این سوال جواب بده: {query}
    پاسخ باید به زبان پارسی باشد.
    از کلمات «پاراگراف»، «متن»، «سند»، «داکیومنت»، «منبع»، یا «بالا» استفاده نکن.
    اگر اطلاعات کافی برای پاسخ وجود نداشت، دقیقاً بگو: «نمی‌توانم جواب شما را بدهم» و هیچ توضیح اضافی نده.
    """

    try:
        response = llm.invoke(prompt)
        response_text = response.content.strip()
    except Exception:
        return "نمی‌توانم جواب شما را بدهم"

    forbidden_words = ["پاراگراف", "متن", "سند", "داکیومنت", "منبع", "بالا"]
    if any(word in response_text for word in forbidden_words) or ("نمی‌توانم" in response_text and response_text != "نمی‌توانم جواب شما را بدهم"):
        return "نمی‌توانم جواب شما را بدهم"
    return response_text

extract_text = StructuredTool.from_function(
    func=extract_text_func,
    name="extract_text",
    description="استخراج اطلاعات از داکیومنت برای سوالات درباره املاک، سایت دلتا، یا کاربران سایت (مثل تعداد آگهی‌های مجاز، خطاهای سایت، یا اطلاعات عمومی).",
    args_schema=None
)

def connect_to_support_func(query: str) -> str:
    """
    دادن شماره پشتیبانی سایت برای سوالاتی که در داکیومنت پاسخی ندارند.
    """
    return f" برای اطلاعات بیشتر، لطفاً با شماره پشتیبانی سایت دلتا (021-8686) تماس بگیرید."

connect_to_support = StructuredTool.from_function(
    func=connect_to_support_func,
    name="connect_to_support",
    description="دادن شماره پشتیبانی سایت دلتا (021-8686) برای سوالات درباره املاک یا سایت که در داکیومنت پاسخی ندارند.",
    args_schema=None
)

def register_property_ad_func(query: str) -> str:
    """
    راهنمایی کاربر برای ثبت آگهی فروش یا اجاره ملک در سایت دلتا.
    """
    return """برای فروش یا اجاره ملک، میتوانید آگهی خود را در سایت دلتا ثبت کنید. برای ثبت آگهی خود مراحل زیر را طی کنید:
    ابتدا وارد حساب کاربری خود شوید یا ثبت نام کنید، سپس با انتخاب گزینه ثبت آگهی فروشی یا ثبت آگهی اجاره و تکمیل فرم مربوطه آگهی خود را در سایت دلتا ثبت کنید.
    در هنگام ثبت ملک سعی کنید تا اطلاعات درخواستی ملک خود را به طور دقیق و کامل وارد نمایید تا اثربخشی آگهی شما افزایش یابد.
    همچنین توجه داشته باشید که ثبت عکس برای آگهی امکان دیده شدن آن را تا 10 برابر افزایش می دهد.
    """

register_property_ad = StructuredTool.from_function(
    func=register_property_ad_func,
    name="register_property_ad",
    description="راهنمایی برای ثبت آگهی فروش یا اجاره ملک وقتی کاربر صراحتاً قصد فروش یا اجاره دارد.",
    args_schema=None
)

llm = base_llm

llm_with_tools = llm.bind_tools([search_properties, extract_text, connect_to_support, register_property_ad], parallel_tool_calls=False)

def get_answer(user_input: str) -> str:
    user_input = normalize_query(user_input)

    sys_msg = SystemMessage(content="""
    تو یک مشاور املاک هستی که با چهار ابزار زیر کار می‌کنی:
    - search_properties: برای وقتی که کاربر می‌خواهد ملکی بخرد (مثل 'یه آپارتمان در تهران بخرم'). فقط وقتی کاربر صراحتاً قصد خرید دارد و مشخصاتی مثل شهر یا نوع ملک داده، این ابزار را انتخاب کن.
    - register_property_ad: برای وقتی که کاربر می‌خواهد ملکی را بفروشد یا اجاره دهد (مثل 'می‌خوام زمینم رو بفروشم' یا 'ملکم رو اجاره بدم'). فقط وقتی کاربر صراحتاً قصد فروش یا اجاره دارد، این ابزار را انتخاب کن.
    - extract_text: برای سوالات درباره املاک، سایت دلتا، یا کاربران سایت (مثل 'چنتا آگهی تو ماه میشه گذاشت'، 'خوش آب و هوا ترین محله‌ها'، 'نحوه ثبت آگهی'، یا خطاهای سایت). اگر سوال درباره تعداد آگهی‌های مجاز، اطلاعات سایت، یا اطلاعات عمومی املاک است، این ابزار را انتخاب کن.
    - connect_to_support: فقط وقتی سوال درباره املاک یا سایت دلتا است اما ابزار extract_text پاسخی پیدا نکرد (مثل 'چطور با مشاور صحبت کنم' بدون اطلاعات در داکیومنت).
    اگر سوال کاربر به املاک یا سایت دلتا مربوط نیست (مثل 'قیمت دلار چنده؟')، هیچ ابزاری انتخاب نکن و فقط بگو: 'نمی‌توانم پاسخ این سوال را بدهم'.
    ابزار مناسب را انتخاب کن و پاسخ را به پارسی بده. فقط یک ابزار را انتخاب کن و از توضیحات اضافی خودداری کن.
    """)

    messages = [sys_msg, HumanMessage(content=user_input)]

    max_retries = 3
    for attempt in range(max_retries):
        try:
            response = llm_with_tools.invoke(messages)
            print(f"LLM response (attempt {attempt + 1}): {response}")  # Debugging
            break
        except Exception as e:
            print(f"LLM invocation error (attempt {attempt + 1}): {str(e)}")
            if attempt == max_retries - 1:
                return "نمی‌توانم پاسخ این سوال را بدهم. لطفاً دوباره امتحان کنید یا با پشتیبانی تماس بگیرید."
            continue

    if hasattr(response, "tool_calls") and response.tool_calls:
        tool_call = clean_tool_call(response.tool_calls[0])
        tool_name = tool_call["name"]
        tool_args = tool_call["args"]
        print(f"Executing tool: {tool_name} with args: {tool_args}")  # Debugging

        try:
            if tool_name == "search_properties":
                return search_properties.run(tool_input=tool_args)
            elif tool_name == "register_property_ad":
                return register_property_ad.run(tool_input=tool_args)
            elif tool_name == "extract_text":
                result = extract_text.run(tool_input=tool_args)
                if result == "نمی‌توانم جواب شما را بدهم":
                    if any(keyword in user_input.lower() for keyword in ["ملک", "آگهی", "سایت", "دلتا", "محله", "مشاور", "آژانس", "خطا", "مجاز", "تعداد"]):
                        return connect_to_support.run(tool_input={"query": user_input})
                return result
            elif tool_name == "connect_to_support":
                return connect_to_support.run(tool_input=tool_args)
            return "نمی‌توانم پاسخ این سوال را بدهم"
        except Exception as e:
            print(f"Tool execution error: {str(e)}")
            if any(keyword in user_input.lower() for keyword in ["ملک", "آگهی", "سایت", "دلتا", "محله", "مشاور", "آژانس", "خطا", "مجاز", "تعداد"]):
                return connect_to_support.run(tool_input={"query": user_input})
            return "نمی‌توانم پاسخ این سوال را بدهم"

    if response.content:
        parsed_tool_call = parse_content_tool_call(response.content)
        if parsed_tool_call:
            tool_call = clean_tool_call(parsed_tool_call)
            tool_name = tool_call["name"]
            tool_args = tool_call["args"]
            print(f"Executing tool from content: {tool_name} with args: {tool_args}")  # Debugging

            try:
                if tool_name == "search_properties":
                    return search_properties.run(tool_input=tool_args)
                elif tool_name == "register_property_ad":
                    return register_property_ad.run(tool_input=tool_args)
                elif tool_name == "extract_text":
                    result = extract_text.run(tool_input=tool_args)
                    if result == "نمی‌توانم جواب شما را بدهم":
                        if any(keyword in user_input.lower() for keyword in ["ملک", "آگهی", "سایت", "دلتا", "محله", "مشاور", "آژانس", "خطا", "مجاز", "تعداد"]):
                            return connect_to_support.run(tool_input={"query": user_input})
                    return result
                elif tool_name == "connect_to_support":
                    return connect_to_support.run(tool_input=tool_args)
                return "نمی‌توانم پاسخ این سوال را بدهم"
            except Exception as e:
                print(f"Tool execution error from content: {str(e)}")
                if any(keyword in user_input.lower() for keyword in ["ملک", "آگهی", "سایت", "دلتا", "محله", "مشاور", "آژانس", "خطا", "مجاز", "تعداد"]):
                    return connect_to_support.run(tool_input={"query": user_input})
                return "نمی‌توانم پاسخ این سوال را بدهم"

    if not any(keyword in user_input.lower() for keyword in ["ملک", "آگهی", "سایت", "دلتا", "محله", "مشاور", "آژانس", "خطا", "مجاز", "تعداد", "بخرم", "بفروشم", "اجاره"]):
        return "نمی‌توانم پاسخ این سوال را بدهم"

    return response.content if response.content else "نمی‌توانم پاسخ این سوال را بدهم"

In [ ]:
# Gradio interface
input = gr.Textbox(label="سوال خود را درباره خرید، فروش، یا اجاره ملک بپرسید")
outputs = gr.Textbox(label="")
interface = gr.Interface(
    fn=get_answer,
    inputs=input,
    outputs=outputs,
    title="دستیار هوشمند دلتا",
    clear_btn="پاک کردن",
    allow_flagging="never",
    theme="dark"
)
interface.launch()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/gradio/blocks.py:1116: UserWarning: Cannot load dark. Caught Exception: 404 Client Error: Not Found for url: https://huggingface.co/api/spaces/dark (Request ID: Root=1-681b3f43-0e1922512f5fab153695310e;ffe08f21-c2a3-4d17-ae64-2e2f98ef26a9)

Sorry, we can't find the page you are looking for.
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")
/usr/local/lib/python3.11/dist-packages/gradio/interface.py:415: UserWarning: The 

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6b8e15875667af2b71.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
